In [0]:
filePath = "dbfs:/FileStore/GlobalRetail/bronze_layer/transaction/transaction_snappy.parquet"
df = spark.read.parquet(filePath)
df.show()

In [0]:
from pyspark.sql.functions import to_timestamp, col
new_df = df.withColumn("transaction_date", to_timestamp(col("transaction_date")))
new_df.printSchema()
display(new_df)

In [0]:
from pyspark.sql.functions import current_timestamp
final_df = new_df.withColumn("ingestion_timestamp", current_timestamp())
display(final_df)

In [0]:
spark.sql("use globalretail_bronze")
final_df.write.format("delta").mode("append").saveAsTable("bronze_transactions")

In [0]:
spark.sql("select * from bronze_transactions limit 100").show()

In [0]:
import datetime
archive_folder = "dbfs:/FileStore/GlobalRetail/bronze_layer/transaction/archive/"
archive_filepath = archive_folder + "_" + datetime.datetime.now().strftime("%Y%m%d%H%M%s")
dbutils.fs.mv(filePath, archive_filepath)
print(archive_filepath)